# SDSetupManager
StableDiffusion WebUI の起動を管理します。[(GitHub)](https://github.com/bmaltais/kohya_ss)

環境の詳細な設定はEnvValueEditor(EVED)を使ってください。

```
2025/05/14 1.0.0 完成
```


# SDSetupManager

## 1. Logging 定義


In [1]:
# @title a. logging定義
import logging
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s [%(levelname)s] %(name)s: %(message)s",
    force=True
)


## 2. モジュール定義

In [2]:
# @title a. SDSetupManager 定義

# ────────────────────────────────────────────────────────────────
# 各モジュール用logger設定
import logging
logger = logging.getLogger(__name__)
# ────────────────────────────────────────────────────────────────

try:
    from an_envmanager import EnvManager
except ImportError:
    get_ipython().system( "pip install  --upgrade an_EnvManager" )
    from an_envmanager import EnvManager

try:
    from an_abracadabra import FileSymlinksUtility, FolderSymlinksUtility
except ImportError:
    get_ipython().system( "pip install --upgrade an_Abracadabra" )
    from an_abracadabra import FileSymlinksUtility, FolderSymlinksUtility

try:
    from an_toolkit import ToolKit
except ImportError:
    get_ipython().system( "pip install  --upgrade an_ToolKit" )
    from an_toolkit import ToolKit

try:
    from an_easyven import EasyVen
except ImportError:
    get_ipython().system( "pip install  --upgrade an_EasyVen" )
    from an_easyven import EasyVen

get_ipython().system( "pip install virtualenv" )
import os, sys, subprocess, certifi, shutil, virtualenv
from pathlib import Path

get_ipython().system( "virtualenv -p /usr/bin/python3 /content/cEnv/venv/sd_env" )



class SDSetupManager:
    def __init__( self, venv_name = "sd_env" ):
        """
        Stable Diffusion WebUI 起動マネージャー
        args:
            venv_name( str ): 仮想環境名
        """
        self.venv_name = venv_name
        logger.info("Initializing SDSetupManager")

        # 仮想環境のセットアップ
        self.even = EasyVen()
        ( self.bch_path,
          self.dst_path,
          self.cur_path,
          self.src_path,
          self.arc_path,
          self.arz_path,
          self.lcl_path,
          self.bin_path,
          self.lib_path,
          self.tmp_path,
          self.org_path,
          self.orz_path,
          self.enva_path,
          self.envb_path,
          self.uni_path ) = self.even.setup(venv_name=self.venv_name)

        # シンボリックリンクと環境変数マネージャ
        self.envmanager             = EnvManager(env_files=[ "settings_ven.env, settings_sdm.env" ])
        # 各種ツール初期化
        self.repo_url_sd            = "https://github.com/AUTOMATIC1111/stable-diffusion-webui.git"
        self.target_dir_sd          = Path( self.envmanager.get_env_var( "Ven_app_folder" ) ) / self.envmanager.get_env_var("Sdm_repo_folder")
        self.req_path_sd            = Path( self.target_dir_sd ) / "requirements.txt"
        self.req_path_sd_versions   = Path( self.target_dir_sd ) / "requirements_versions.txt"
        self.python_path            = self.bin_path / "python"
        self.stable_tag             = "v1.10.1"
        self.toolkit                = ToolKit()

    def launch_webui(self, update: bool = False):
        """
        StanleDiffusion WebUI を起動します
        args
            update( bool ): True なら既存リポジトリを pull してから起動
        """
        logger.info("🐎🐎🐎🐎🐎 Launching StableDiffusion WebUI 🐎🐎🐎🐎🐎")
        self.create_and_configure_venv()
        if update:
            self.update_repository()
        else:
            self.clone_repository()
        self.recreate_symlink(link_path = self.tmp_path.joinpath( self.envmanager.get_env_var( "Sdm_repo_folder" ) ),
                              target_path = self.target_dir_sd )
        self.configure_environment_variables()
        options = self.configure_launch_options() + self.detect_gpu_and_configure()
        launch_cmd = ' '.join(options)
        logger.info(f"Final launch command: {launch_cmd}")
        logger.info( f"target_dir_sd :{ self.target_dir_sd }")

        os.chdir(self.target_dir_sd)
        get_ipython().system( launch_cmd )

    def create_and_configure_venv(self):
        """
        - virtualenv でローカルに venv を作成
        - starlinka_env_setup.sh を source して
          キャッシュ用の環境変数を設定
        """
        # ① ローカル venv のパスを組み立て
        local_venv_path = Path(self.envmanager.get_env_var("Ven_local_venv_folder")) / self.venv_name / self.bch_path

        # ② ToolKit 経由で仮想環境を作成
        self.toolkit.create_virtualenv(
            venv_path=local_venv_path,
            python_executable="/usr/bin/python3"
        )

        # ③ キャッシュ用 env 設定スクリプトを source
        #    注意：source は bash -c 経由で呼び出す
        Path(local_venv_path).parent.mkdir(parents=True, exist_ok=True)
        # cmd = f"bash -c 'source {self.uni_path} {self.venv_name}'"
        cmd = f"bash -c 'cd /content && source {self.uni_path} {self.venv_name}'"
        logger.info(f"running: {cmd}")
        self.toolkit.executor_sync(cmd=cmd, shell=True)

    def update_repository(self):
        """
        リポジトリをアップデートします
        """
        repo_dir = self.target_dir_sd
        if not (repo_dir/" .git").exists():
            return self.clone_repository()

        logger.info("Checking for updates…")
        cwd = os.getcwd()
        os.chdir(str(repo_dir))
        try:
            # リモートの最新情報だけ先に取ってくる
            self.toolkit.executor_sync(cmd=["git","remote","update"], shell=False)
            local = subprocess.check_output(["git","rev-parse","@"], cwd=repo_dir).strip()
            remote = subprocess.check_output(["git","rev-parse","@{u}"], cwd=repo_dir).strip()
            if local == remote:
                logger.info("Already up to date.")
                return
            # あるなら fast-forward だけで取り込む
            logger.info("Pulling latest changes (fast-forward)…")
            self.toolkit.executor_sync(cmd=["git","pull","--ff-only"], shell=False)
            logger.info("Updating submodules (shallow)…")
            self.toolkit.executor_sync(cmd=[
                "git","submodule","update","--init","--recursive","--depth","1"
            ], shell=False)
            logger.info("Repository update 完了")
        finally:
            os.chdir(cwd)

    def clone_repository(self):
        """
        リポジトリクローン＆サブモジュール更新
        """
        if (self.target_dir_sd / ".git").exists():
            logger.info("Repository already cloned, skipping clone")
            return
        parent_dir = self.target_dir_sd.parent              # リポジトリのクローニングには、親フォルダが存在していることが必要
        if not parent_dir.exists():
            parent_dir.mkdir(parents=True, exist_ok=True)
        if self.target_dir_sd.exists():
            shutil.rmtree(self.target_dir_sd)
        self.target_dir_sd.mkdir( parents=True, exist_ok=True )
        # クローンコマンド生成
        if self.stable_tag is not None:
            cmd = [
                "git", "clone",
                "--branch", self.stable_tag,
                "--depth", "1",
                "--recursive",
                str( self.repo_url_sd ), str(self.target_dir_sd)
            ]
        else:
            cmd = ["git", "clone", "--recursive", str( self.repo_url_sd ), str(self.target_dir_sd)]

        logger.info(f"Running git clone: {' '.join(cmd)}")
        # 実行
        self.toolkit.executor_sync(cmd=cmd, shell=False)
        logger.info("Repository cloned successfully.")
        # サブモジュール更新
        cwd = os.getcwd()
        os.chdir(str(self.target_dir_sd))
        subm_cmd = ["git", "submodule", "update", "--init", "--recursive"]
        self.toolkit.executor_sync(cmd=subm_cmd, shell=False)
        logger.info("Submodules updated successfully.")
        os.chdir(cwd)

    def recreate_symlink( self, link_path: Path, target_path: Path, is_dir: bool = True):
        """
        link_path（リンクを置きたいパス）に既存のリンク・フォルダ・ファイルがあれば削除し、
        target_path への新しいシンボリックリンクを張り直す。

        :param link_path: Path — シンボリックリンクを置くパス
        :param target_path: Path — リンク先（既存のファイル／ディレクトリ）
        :param is_dir: bool — target がディレクトリなら True
        """
        # 既存の “何か” があるなら削除
        if link_path.exists() or link_path.is_symlink():
            if link_path.is_symlink():
                link_path.unlink()
                logger.info(f"既存のシンボリックリンクを削除: {link_path}")
            elif link_path.is_dir():
                shutil.rmtree(link_path)
                logger.info(f"既存のディレクトリを再帰削除: {link_path}")
            else:
                link_path.unlink()
                logger.info(f"既存のファイルを削除: {link_path}")

        # 親フォルダを保証
        link_path.parent.mkdir(parents=True, exist_ok=True)

        # シンボリックリンクを作成
        try:
            link_path.symlink_to(target_path, target_is_directory=is_dir)
            logger.info(f"シンボリックリンクを作成: {link_path} → {target_path}")
        except OSError as e:
            logger.warning(f"シンボリックリンクに失敗({e.errno})、コピーにフォールバック: {link_path}")
            if is_dir:
                shutil.copytree(target_path, link_path)
            else:
                shutil.copy2(target_path, link_path)
            logger.info(f"コピー完了: {target_path} → {link_path}")


    def configure_launch_options(self):
        logger.info("Configuring launch options")
        return [
            str(self.bin_path / 'python'),
            str(self.target_dir_sd / 'launch.py'),
            "--ckpt-dir", str(self.target_dir_sd / "models/Stable-diffusion"),
            "--share", "--no-half", "--no-half-vae", "--xformers",
            "--enable-insecure-extension-access"
        ]

    def detect_gpu_and_configure(self):
        logger.info("Detecting GPU")
        gpu_name = "Unknown"
        try:
            result = subprocess.check_output("nvidia-smi -L", shell=True).decode()
            gpu_name = result.strip().split(":")[1].split("(")[0].strip()
        except Exception:
            pass
        logger.info(f"Detected GPU: {gpu_name}")

        options = ["--listen"]
        if "T4" in gpu_name:
            options.append("--opt-split-attention")
        elif any(x in gpu_name for x in ("K80","L4")):
            options.append("--medvram")
        elif any(x in gpu_name for x in ("A100","V100")):
            options.append("--opt-sdp-attention")
        logger.info(f"Additional options: {options}")
        return options

    def configure_environment_variables(self):
        logger.info("Setting environment variables")
        # この環境変数を指定しておくと、安全かつ最新の CA リストで、外部からモデルやデータをダウンロードできるようになるらしい
        os.environ['SSL_CERT_FILE'] = certifi.where()
        os.environ["OMP_NUM_THREADS"] = "1"
        os.environ["MKL_NUM_THREADS"] = "1"
        os.environ["TRANSFORMERS_VERBOSITY"] = "info"
        os.environ["HF_HUB_DISABLE_PROGRESS_BARS"] = "true"
        os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "garbage_collection_threshold:0.9,max_split_size_mb:512"
        os.environ["MPLBACKEND"] = "agg"
        self.env = {}
        self.env.update({
            "PYTORCH_CUDA_ALLOC_CONF": os.environ["PYTORCH_CUDA_ALLOC_CONF"],
            "MPLBACKEND": os.environ["MPLBACKEND"],
        })
        logger.info("Environment variables configured")





Mounted at /content/drive
created virtual environment CPython3.11.12.final.0-64 in 350ms
  creator CPython3Posix(dest=/content/cEnv/venv/sd_env, clear=False, no_vcs_ignore=False, global=False)
  seeder FromAppData(download=False, pip=bundle, setuptools=bundle, via=copy, app_data_dir=/root/.local/share/virtualenv)
    added seed packages: pip==25.1.1, setuptools==80.3.1
  activators BashActivator,CShellActivator,FishActivator,NushellActivator,PowerShellActivator,PythonActivator


## 3. 実行

In [ ]:
# @title a. MainLoop 実行

# ────────────────────────────────────────────────────────────────
# 各モジュール用logger設定
import logging
logger = logging.getLogger(__name__)
logger.addHandler(logging.NullHandler())
# ────────────────────────────────────────────────────────────────

if __name__ == "__main__":
    sd = SDSetupManager(venv_name="sd_env")
    # update=True にすると既存リポジトリを pull してから起動
    sd.launch_webui(update=True)


